# Item 48: Validate Subclasses with `__init_subclass__`

In Python, a *metaclass* is defined by inherting from `type`. In the defualt case, a metaclass receives the contents of associated `class` statements in its `__new__` method. Using metaclasses for validation can raise errors much earlier, even before the class `__init__` method is called.

In [1]:
# Here, we can inspect and modify the class information before the type is actually constructed
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print(f'* Running {meta}.__new__ for {name}')
        print('Bases:', bases)
        print(class_dict)
        return type.__new__(meta, name, bases, class_dict)

class MyClass(metaclass=Meta):
    stuff = 123

    def foo(self):
        pass

class MySubclass(MyClass):
    other = 567

    def bar(self):
        pass

* Running <class '__main__.Meta'>.__new__ for MyClass
Bases: ()
{'__module__': '__main__', '__qualname__': 'MyClass', 'stuff': 123, 'foo': <function MyClass.foo at 0x7f9a68593790>}
* Running <class '__main__.Meta'>.__new__ for MySubclass
Bases: (<class '__main__.MyClass'>,)
{'__module__': '__main__', '__qualname__': 'MySubclass', 'other': 567, 'bar': <function MySubclass.bar at 0x7f9a288013a0>}


We can add functionality to the `Meta.__new__` method in order to validate all of the parameters of an associated class before it's defined.

In [2]:
# Note that it's important not to apply the same validation to the base class
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate subclasses of the Polygon class
        if bases:
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)

class Polygon(metaclass=ValidatePolygon):
    sides = None # Must be specified by subclass

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

class Triangle(Polygon):
    sides = 3

class Rectangle(Polygon):
    sides = 4

class Nonagon(Polygon):
    sides = 9

assert Triangle.interior_angles() == 180
assert Rectangle.interior_angles() == 360
assert Nonagon.interior_angles() == 1260

In [3]:
# Try: polygon with fewer than 3 sides. -> The validation will cause the class to fail immediately
print('Before class')

class Line(Polygon):
    print('Before sides')
    sides = 2
    print('After sides')

print('After class')

Before class
Before sides
After sides


ValueError: Polygons need 3+ sides

Python 3.6 introduced simplified syntax, namely the `__init_subclass__` special method, for achieving the same behavior while avoiding metaclasses entirely.

In [4]:
class BetterPolygon:
    sides = None  # Must be specified by subclass

    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.sides < 3:
            raise ValueError('Polygons need 3+ sides')

    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180

class Hexagon(BetterPolygon):
    sides = 6

assert Hexagon.interior_angles() == 720

In [5]:
# If we define an invalid subclass of BetterPolygon, the same ValueError exception will be raised
print('Before class')

class Point(BetterPolygon):
    sides = 1

print('After class')

Before class


ValueError: Polygons need 3+ sides

Another problem with the metaclass machinery is that we can only specify a single metaclass per class definition.

In [6]:
# Class to validate the fill color used for a region
class ValidateFilled(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate subclasses of the Filled class
        if bases:
            if class_dict['color'] not in ('red', 'green'):
                raise ValueError('Fill color must be supported')
        return type.__new__(meta, name, bases, class_dict)
    
class Filled(metaclass=ValidateFilled):
    color = None  # Must be specified by subclass

In [7]:
# When we try to use the Polygon metaclass and Filled metaclass together, we get a cryptic error message
class RedPentagon(Filled, Polygon):
    color = 'red'
    sides = 5

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [8]:
# Its possible to fix this by creating a complex hierarchy of metaclass type definitions to layer 
# validation
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # Only validate non-root classes
        if not class_dict.get('is_root'):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)


class Polygon(metaclass=ValidatePolygon):
    is_root = True
    sides = None  # Must be specified by subclasses

class ValidateFilledPolygon(ValidatePolygon):
    def __new__(meta, name, bases, class_dict):
        # Only validate non-root classes
        if not class_dict.get('is_root'):
            if class_dict['color'] not in ('red', 'green'):
                raise ValueError('Fill color must be supported')
        return super().__new__(meta, name, bases, class_dict)


class FilledPolygon(Polygon, metaclass=ValidateFilledPolygon):
    is_root = True
    color = None  # Must be specified by subclasses


In [9]:
# The above requires every FilledPolygon instance to be a Polygon instance
class GreenPentagon(FilledPolygon):
    color = 'green'
    sides = 5

greenie = GreenPentagon()
assert isinstance(greenie, Polygon)

In [10]:
# Validation works for colors
class OrangePentagon(FilledPolygon):
    color = 'orange'
    sides = 5

ValueError: Fill color must be supported

In [11]:
# Validation works for number of sides
class RedLine(FilledPolygon):
    color = 'red'
    sides = 2

ValueError: Polygons need 3+ sides

The approach used above ruins composability. If we want to apply color validation logic from `ValidateFilledPolygon` to another hierarchy of classes, we'll have to duplicate all of the logic again, which reduces code reuse and increases boilerplate.

The `__init_subclass__` method can be used to solve this problem. It can be defined by multiple levels of class hierarchy as long as the super built-in function is used to call any parent or sibling `__init_subclass__` definitions. Its even compatible with multiple inheritance

In [12]:
class Filled:
    color = None  # Must be specified by subclasses

    def __init_subclass__(cls):
        super().__init_subclass__()
        if cls.color not in ('red', 'green', 'blue'):
            raise ValueError('Fills need a valid color')

In [13]:
# We can inherrit from both classes to define a new class
class RedTriangle(Filled, Polygon):
    color = 'red'
    sides = 3

ruddy = RedTriangle()
assert isinstance(ruddy, Filled)
assert isinstance(ruddy, Polygon)

In [14]:
# If we specify the number of sides incorrectly, we get a validation error
print('Before class')

class BlueLine(Filled, Polygon):
    color = 'blue'
    sides = 2

print('After class')

Before class


ValueError: Polygons need 3+ sides

In [15]:
# If we specify a color incorrectly, we also get a validation error
print('Before class')


class BeigeSquare(Filled, Polygon):
    color = 'beige'
    sides = 4


print('After class')

Before class


ValueError: Fills need a valid color

In [16]:
# We can even use __init_subclass__ in complex cases like diamond inheritance
class Top:
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Top for {cls}')

class Left(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Left for {cls}')

class Right(Top):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Right for {cls}')


class Bottom(Left, Right):
    def __init_subclass__(cls):
        super().__init_subclass__()
        print(f'Bottom for {cls}')


Top for <class '__main__.Left'>
Top for <class '__main__.Right'>
Top for <class '__main__.Bottom'>
Right for <class '__main__.Bottom'>
Left for <class '__main__.Bottom'>
